In [ ]:
dataset_name = "colon-cancer"
scale_data = False

if dataset_name == "a9a":
    url_train = f"https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/{dataset_name}"

    trainset = urllib.request.urlopen(url_train)

    trainX, trainY = load_svmlight_file(trainset)

    # train_data = torch.tensor(normalize(a1a_trainX.toarray(), norm='l2', axis=1), dtype=torch.float, device=device)
    train_data = torch.tensor(trainX.toarray(), dtype=torch.float, device=device)
    train_target = torch.tensor(trainY, dtype=torch.float, device=device)

elif dataset_name == "colon-cancer":
    # with open(f"datasets/{dataset_name}", "r") as f_:
    #     filelines = f_.readlines()

    # targets = []
    # dataset = []

    # for line in filelines:
    #     split_line = line.split(sep=" ")
    #     targets.append(split_line[0])
    #     data = []
    #     for i in split_line[2:]:
    #         data.append(i.split(sep=":")[1])
    #     dataset.append(data)

    # trainX = np.array(dataset, dtype=np.float32)
    # trainY = np.array(targets, dtype=np.float32)

    trainX, trainY = load_svmlight_file(f"datasets/{dataset_name}") 


    train_data = torch.tensor(trainX.toarray(), dtype=torch.float, device=device)
    train_target = torch.tensor(trainY, dtype=torch.float, device=device)

elif dataset_name == "covtype.libsvm.binary.scale" or dataset_name == "covtype.libsvm.binary":
    trainX, trainY = load_svmlight_file(f"datasets/{dataset_name}")

    sample = np.random.choice(trainX.shape[0], round(trainX.shape[0] * 0.05), replace=False)

    assert sample.shape == np.unique(sample).shape

    trainX = trainX[sample]
    trainY = trainY[sample]

    train_data = torch.tensor(trainX.toarray(), dtype=torch.float, device=device)
    train_target = torch.tensor(trainY, dtype=torch.float, device=device)

elif dataset_name == "mushrooms":
    trainX, trainY = load_svmlight_file(f"datasets/{dataset_name}")

    train_data = torch.tensor(trainX.toarray(), dtype=torch.float, device=device)
    train_target = torch.tensor(trainY, dtype=torch.float, device=device)


if scale_data:
    r1 = -10
    r2 = 10
    scaling_vec = (r1 - r2) * torch.rand(train_data.shape[1], device=device) + r2
    scaling_vec = torch.pow(torch.e, scaling_vec)
    train_data = scaling_vec * train_data

train_load = data_utils.TensorDataset(train_data, train_target)
    

# SGD with $lr=1e^{-1}$

In [ ]:
w = torch.zeros(train_data.shape[1], device=device).requires_grad_()
hist_sgd = []

loss = F(w)
g, = torch.autograd.grad(loss, w, create_graph=True)

hist_sgd.append([loss.item(), (torch.linalg.norm(g) ** 2).item()])


for step in range(STEPS):
    
    for i, (batch_data, batch_target) in enumerate(train_dataloader):
        loss = F(w, X=batch_data, y=batch_target)
        g, = torch.autograd.grad(loss, w, create_graph=True)
        precond = eta
        w = w - ( precond * g )

    print(f"Loss: {F(w).item()} | GradNorm^2: {(torch.linalg.norm(g) ** 2).item()}")

    hist_sgd.append([F(w).item(), (torch.linalg.norm(g) ** 2).item()])


In [ ]:
def diag_estimate_old(data_shape_cols, grad, weights, iters):
    Ds = []
    for j in range(iters):
        z = rademacher(data_shape_cols).to(device=device)
        with torch.no_grad():
            hvp = torch.autograd.grad(grad, weights, grad_outputs=z, retain_graph=True)[0]
        Ds.append((hvp*z))

    return torch.mean(torch.stack(Ds), 0)

def diag_estimate(weights, iters):
    Ds = []
    for j in range(iters):
        z = rademacher(weights.shape[0]).to(device=weights.get_device())
        with torch.no_grad():
            hvp = torch.autograd.grad(weights.grad, weights, grad_outputs=z, retain_graph=True)[0]
        Ds.append((hvp*z))

    return torch.mean(torch.stack(Ds), 0)

In [ ]:
w = torch.zeros(train_data.shape[1], device=device).requires_grad_()

alpha = 1e-1
beta = 0.999

loss = F(w)
g, = torch.autograd.grad(loss, w, create_graph=True)

hist_sps_d = [[loss.item(), (torch.linalg.norm(g) ** 2).item()]]

Dk = diag_estimate_old(w.shape[0], g, w, 100)

for step in range(STEPS):
    
    loss = F(w)
    g, = torch.autograd.grad(loss, w, create_graph=True)
    hist_sps_d.append([F(w).item(), (torch.linalg.norm(g) ** 2).item()])

    print(f"Loss: {loss.item()} | GradNorm^2: {(torch.linalg.norm(g) ** 2).item()}")

    
    for i, (batch_data, batch_target) in enumerate(train_dataloader):
            
        # print(f"Loss: {loss.item()} | GradNorm^2: {(torch.linalg.norm(g) ** 2).item()}")

        # if i%1 == 0: print(f"Accuracy: {np.mean(trainX.dot(w.cpu().data.numpy())*trainY > 0)}")

        loss = F(w, X=batch_data, y=batch_target)
        g, = torch.autograd.grad(loss, w, create_graph=True)

        vk = diag_estimate_old(w.shape[0], g, w, 1)

        # Smoothing and Truncation        
        Dk = beta * Dk + (1 - beta) * vk
        Dk_hat = torch.abs(Dk)
        Dk_hat[Dk_hat < alpha] = alpha
        Dk_hat_inv = 1 / Dk_hat

        gnorm = (g * Dk_hat_inv).dot(g)
        
        if gnorm.item() < 1e-13:
            continue
        precond = (loss / (gnorm)) * Dk_hat_inv
        
        with torch.no_grad():
            w.sub_(precond * g)

Loss: 0.6931473016738892 | GradNorm^2: 1.7407530546188354
Loss: 0.0 | GradNorm^2: 0.0
Loss: 0.0 | GradNorm^2: 0.0
Loss: 0.0 | GradNorm^2: 0.0
Loss: 0.0 | GradNorm^2: 0.0
Loss: 0.0 | GradNorm^2: 0.0
Loss: 0.0 | GradNorm^2: 0.0


KeyboardInterrupt: 

In [ ]:

class SPSD(Optimizer):
    def __init__(self, params, preconditioner="none", default_kwargs=None):
        defaults = dict()
        if default_kwargs:
            defaults.update(**default_kwargs)

        self.preconditioner = None

        if preconditioner == "hutch":
            self.preconditioner = Hutch()
            
        self.preconditioner_initialized = False

        # TO-DO: Think of something better
        self.replay_buffer = []

        super().__init__(params, defaults)

    def step(self, closure=None):
        for group in self.param_groups:
            weights = list(group["params"])
            
            loss = closure()
            grad = torch.autograd.grad(loss, weights, create_graph=True)
            loss = loss.item()

            if self.preconditioner:
                if not self.preconditioner_initialized:
                    self.preconditioner.init(self, grad, 100)
                    self.preconditioner_initialized = True
                self.preconditioner.step(self, grad, 1)
            else:
                self.init_empty_precond()
                self.preconditioner_initialized = True
            
            self.update(grad, loss)
                          
        return loss

    def init_empty_precond(self):
        for group in self.param_groups:
            for p in group['params']:
                self.state[p]['Dk'] = torch.ones_like(p) 
                self.state[p]['DkhatInv'] = torch.ones_like(p)

    @torch.no_grad()
    def update(self, grad, loss):
        for group in self.param_groups: 
            gnorm_square = 0.
            for p, g in zip(group['params'], grad):
                g_detached = g.detach().clone()
                self.state[p]['scaled_grad'] = self.state[p]['DkhatInv'].mul(g_detached)
                gnorm_sq = self.state[p]['scaled_grad'].mul(g_detached).sum()
                gnorm_square += gnorm_sq

            if gnorm_square.item() < 1e-13:
                return 
            step_size = loss / gnorm_square

            for p in group['params']:
                p.sub_(self.state[p]['scaled_grad'].mul(step_size))

            self.replay_buffer.append({
                "loss" : loss,
                "grad_norm_sq" : gnorm_square.item(),
            })
        